In [1]:
(
exec 9<&1

echopat='*echo $?*'

# The code below seems to work, but it may be sensitive to
# the timeout and sleep values.  Hopefully it works well enough
# until we have time to modify the bash_kernel code.

blockingread()
{
   while sleep 0.2; do
      IFS= read -r ln
      if [[ "$ln" == $echopat ]]; then
         # give Jupyter a return code when it asks for one
         echo "0" >&9
         echo -n "$PS1" >&9
         continue
      fi
      [ "$ln" = "" ] && continue
      break
   done
}

readcell() {
    blockingread
    echo "$ln"
    echo -n "$PS2">&9
    # read rest of the cell contents
    while IFS= read -t 0.2 -r lnr; do
      ## somehow get make this next line unnecessary!!!
      [[ "$lnr" == *endofcell* ]] && break
      echo "$lnr"
      # The problem is this next line.  Jupyter will not send the next line unless
      # PS2 is sent.  But if there are no more lines, then Jupyter will block.
      # Therefore, it is necessary to modify the bash_kernel code to remove the
      # requirement for the endofcell marker.
      echo -n "$PS2">&9
    done
}
   
echo "Ready."
echo -n "$PS1"

basedir="$(pwd)"
new_job_dir()
{
    mkdir -p "$basedir/jobs"
    jobn=1000
    while [ -d "$basedir/jobs/j-$jobn" ]; do
        (( jobn++ ))
        sleep 0.2 # use only when debugging
    done
    echo "$basedir/jobs/j-$jobn"
}

# for testing:
while sleep 0.2; do
    echo $$ >>/tmp/sbash.log
    ls -l /proc/$BASHPID/fd >>/tmp/sbash.log
    input="$(readcell)"
    if [[ "$input" = *quit* ]]; then
      echo "bye"
      # the PS1 will be sent by the parent bash (the one created by jupyter)
      break
    fi
    jd="$(new_job_dir)"
    echo "$input" >"$jd/input"
    bash "$jd/input" >"$jd/output" 2>&1
    echo "$?" >"$jd/rc"
    echo "$(cat "$jd/output")"
    echo -n "$PS1"
done
)

Ready.


In [2]:
date
endofcell

bash: /home/centos/notebooks/jobs/j-1000/input: No such file or directory
bash: /home/centos/notebooks/jobs/j-1000/output: No such file or directory
bash: /home/centos/notebooks/jobs/j-1000/rc: No such file or directory
cat: /home/centos/notebooks/jobs/j-1000/output: No such file or directory



In [4]:
line one
line two
endofcell

begin test echo of cell contents:
line one
line two
end


In [5]:
one
endofcell

begin test echo of cell contents:
one
end


In [6]:
quit
endofcell

bye
